In [ ]:
import numpy as np
import hoomd
import gsd.hoomd
import itertools
 
num_particles = 1000
density = 0.5
 
box_volume = num_particles / density
L = box_volume ** (1 / 2.0)
 
N = int(np.ceil(num_particles ** (1.0 / 2.0)))
x = np.linspace(-L / 2, L / 2, N, endpoint=False)
 
particle_spacing = 1.0
if x[1] - x[0] < particle_spacing:
    raise RuntimeError('density too high to initialize on square lattice')
 
position_2d = list(itertools.product(x, repeat=2))[:num_particles]
 
# create snapshot
device = hoomd.device.CPU()
 
snap = hoomd.Snapshot()
 
snap.particles.N = num_particles
snap.particles.types = ['A']
snap.configuration.box = [L, L, 0, 0, 0, 0]
snap.particles.position[:, 0:2] = position_2d
snap.particles.typeid[:] = [0] * num_particles
 
# Use hard sphere Monte-Carlo to randomize the initial configuration
mc = hoomd.hpmc.integrate.Sphere(default_d=0.01)
mc.shape['A'] = dict(diameter=1.0)
 
sim = hoomd.Simulation(device=device, seed=1)
sim.create_state_from_snapshot(snap)
sim.operations.integrator = mc
 
RANDOMIZE_STEPS = 10000
device.notice('Randomizing initial state...')
sim.run(RANDOMIZE_STEPS)
device.notice(f'Move counts: {mc.translate_moves}')
device.notice('Done.')
 
hoomd.write.GSD.write(
state=sim.state, filename='hard_disk_initial_state.gsd', mode='wb'
)

In [ ]:
snapshot = sim.state.get_snapshot()
 
pos = snapshot.particles.position
x,y = pos[:,0], pos[:,1]
 
import matplotlib.pyplot as plt
 
plt.scatter(x,y)
plt.axis('equal')